In [1]:
import instelec as ie
u = ie.ureg # lida com as unidades

## Motores e grupos

Abaixo definiremos os motores que serão usados. Cada motor recebe um triângulo de potência e um número de fases.

In [2]:
eng1 = ie.Engine(ie.PowerTriangle(8*u.kilovolt_ampere, 0.8), 1)
eng2 = ie.Engine(ie.PowerTriangle(35*u.kilovolt_ampere, 0.7), 3)
eng3 = ie.Engine(ie.PowerTriangle(11*u.kilovolt_ampere, 0.8), 1)
eng4 = ie.Engine(ie.PowerTriangle(26*u.kilovolt_ampere, 0.7), 3)
eng5 = ie.Engine(ie.PowerTriangle(17.2*u.kilovolt_ampere, 0.7), 1)
eng6 = ie.Engine(ie.PowerTriangle(28*u.kilovolt_ampere, 0.7), 3)
eng7 = ie.Engine(ie.PowerTriangle(12.2*u.kilovolt_ampere, 0.8), 1)

Cada grupo recebe um dicionário com motores e o número de vezes que aquele motor aparece no grupo.

In [3]:
ccm1 = ie.EngineGroup({eng1: 2, eng2: 2, eng3: 1})
ccm2 = ie.EngineGroup({eng4: 3, eng3: 1, eng5: 1})
qdtl = ie.EngineGroup({eng1: 1, eng6: 1, eng7: 3})

## Cálculo da seção dos condutores do CCM2.

In [4]:
ccm2_section = ie.CupperPVC(
    'B1',
    ccm2.power.power_factor,
    ccm2.phase_num
)

Ao chamar o método .grouping_correction(num), o fator de correção por agrupamento multiplica a tabela de ampacidade.

In [5]:
ccm2_section.grouping_correction(5)

Material: cupper
Insulator: PVC
Instalation method: B1
Power factor: 0.7130175825070523
Phase number: 3

Seção obtida pelo método da ampacidade.

In [6]:
amperage_section = ccm2_section.by_amperage(ccm2.charge_current())
amperage_section

240.0 <Unit('millimeter ** 2')>

Seção obtida pelo método da queda de tensão.

In [9]:
voltage_drop_section = ccm2_section.by_voltage_drop(ccm2.charge_current(), 29*u.meter, 0.03)
voltage_drop_section

25 <Unit('millimeter ** 2')>

Seção obtida pelo método do curto circuito.

In [10]:
short_circuit_section = ccm2_section.by_short_circuit(5*u.kiloampere, 0.01*u.second)
short_circuit_section

6.0 <Unit('millimeter ** 2')>

O condutor de fase deve ser o maior dos condutores obtidos anteriormente.

In [11]:
phase_condutor_section = max(
    amperage_section, voltage_drop_section, short_circuit_section)

In [12]:
protection_condutor = ccm2_section.protection_condutor(phase_condutor_section)
protection_condutor

120.0 <Unit('millimeter ** 2')>

## Triângulo de potência geral

In [13]:
total_power = ccm1.power + ccm2.power + qdtl.power

O fator de potência pode ser chamado a partir do triângulo de potência.

In [14]:
print(total_power.power_factor)

0.7326194206715165


O método power_factor_to retorna os valores de potência reativa que devem ser adicionados para chegar a um fator de potência, respectivamente, capacitivo e indutivo igual ao valor dado.

In [15]:
inf, sup = total_power.power_factor_to(0.92)
print(inf, sup, sep='\n')

-234.1844847964845 kilovolt_ampere_reactive
-86.93905338543757 kilovolt_ampere_reactive
